In [7]:
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 

sys.path.append(os.getcwd())
import tsa_utils
import deep_cnn
import mini_cnn

from shutil import copytree
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
sys.path.append(os.getcwd())
import tsa_utils

import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 
import sklearn
from sklearn import metrics

sys.path.append(os.getcwd())
import tsa_utils
import mini_cnn
import pip
import numpy as np
import matplotlib as mpl
import os
import matplotlib.pyplot as plt
import matplotlib.animation
import tensorflow as tf
from numpy import genfromtxt
import pandas as pd
import sys
import math 
import sklearn
from sklearn import metrics

sys.path.append(os.getcwd())
import tsa_utils
import mini_cnn

pip.main(['install', '--upgrade', 'dask'])


Requirement already up-to-date: dask in /usr/local/lib/python2.7/site-packages


0

In [10]:
copytree("/models/multi-class8/multi_class", "/output/multi_class")

In [11]:
#Model parameters
BATCH_SIZE=10
FILTER_COUNT=12
KERNEL_SIZE1=(1,6,6)
DEPTHSTRIDE=1
XSTRIDE=1
YSTRIDE=1
POOLSIZE1=(16,3,3)
POOLSIZE2=(3,3)
POOL_STRIDES1=(1,2,2)
POOL_STRIDES=(1,1)
STEPS=3000
XSIZE=392
#XSIZE=512
#XSIZE=270
#YSIZE=340
YSIZE=660
LEARNING_RATE=0.01
EPSILON=0
IMAGE_DEPTH=16
CHANNELS=1
FLAT_POOL_SIZE=46656

DATA_PATH="/data_volume/"
CHECKPOINT_PATH="/output/multi_class"
MODEL_ID="multi_class"

tf.logging.set_verbosity(tf.logging.INFO)

slice_locations = {1:("top","right"),
	2:("middle","right"), 
	3:("top","left"),
	4:("middle","left"),
	5:("top","middle"),
	6:("top","right"),
	7:("top","left"),
	8:("middle","right"),
	9:("middle","middle"),
	10:("middle","left"),
	11:("bottom","right"),
	12:("bottom","left"),
	13:("bottom","right"),
	14:("bottom","left"),
	15:("bottom","right"),
	16:("bottom","left"),
	17:("top","middle")}

In [70]:
class ZoneModel():

    def __init__(self, model_id, ids, x_slice, y_slice, data_path, labels, checkpoint_path=".", localize=False):
        self.model_id = model_id
        self.ids = ids
        self.x_slice = x_slice
        self.y_slice = y_slice
        self.data_path = data_path
        self.checkpoint_path = checkpoint_path
        self.labels = labels
        self.localize = localize

    def build_model(self, data, labels, mode):
        if mode == tf.contrib.learn.ModeKeys.INFER:
            BATCH_SIZE=1
        else:
            BATCH_SIZE=10
        print(XSIZE, YSIZE)
        data = tf.reshape(data, [BATCH_SIZE, IMAGE_DEPTH, YSIZE, XSIZE, CHANNELS])
        conv1 = tf.layers.conv3d(inputs=data, filters=FILTER_COUNT, kernel_size=KERNEL_SIZE1, padding="same", 
                strides=(DEPTHSTRIDE,XSTRIDE,YSTRIDE), name="conv1", trainable=not self.localize)
        print(conv1)
        conv2 = tf.layers.conv3d(inputs=conv1, filters=FILTER_COUNT, kernel_size=(3,3,3), padding="same",
                strides=(1, 2, 2), name="conv2", activation=tf.nn.relu, trainable=not self.localize)
        print(conv2)
        pool1 = tf.layers.max_pooling3d(inputs=conv2, pool_size=POOLSIZE1, strides=POOL_STRIDES1, name="pool1")
        print(pool1)
        flattener = tf.reshape(pool1, [BATCH_SIZE, 164, 97, 12])
        pool2 = tf.layers.max_pooling2d(inputs=flattener, pool_size=(3,3), strides=(2,2), name="pool2")
        print(pool2)
        conv3 = tf.layers.conv2d(inputs=pool2, filters=FILTER_COUNT, kernel_size=(3,3), padding="same", 
                strides=(XSTRIDE,YSTRIDE), name="conv3", trainable=not self.localize)
        print(conv3)
        pool3 = tf.layers.max_pooling2d(inputs=conv3, pool_size=(2,2), strides=POOL_STRIDES, name="pool1")
        print(pool3)
        conv4 = tf.layers.conv2d(inputs=pool3, filters=FILTER_COUNT, kernel_size=(3,3), padding="same", 
                strides=(1, 1), name="conv4", activation=tf.nn.relu, trainable=not self.localize)
        print(conv4)
        pool4 = tf.layers.max_pooling2d(inputs=conv4, pool_size=POOLSIZE2, strides=POOL_STRIDES, name="pool2", padding="same")
        print(pool4)
        flat_pool = tf.reshape(pool2, [BATCH_SIZE, FLAT_POOL_SIZE])
        flat_pool=tf.identity(flat_pool, name="flat_pool")

        logits = tf.layers.dense(inputs=flat_pool, units=17, trainable=True)
        logits = tf.identity(logits, name="logits")
        logits = tf.reshape(logits, [BATCH_SIZE,17])
        predictions = {
            "classes": tf.argmax(
              input=logits, axis=1, name="classes"),
          "probabilities": tf.nn.sigmoid(
              logits, name="prob_tensor")}

        if mode == tf.contrib.learn.ModeKeys.TRAIN:
            #flat_labels = tf.reshape(labels, [BATCH_SIZE, 17])
            labels=tf.identity(labels, name="labels")
            #class_weights=tf.reduce_sum(tf.multiply(flat_labels, tf.constant(WEIGHTS, dtype=tf.int64)), axis=1)
            #print(class_weights)
            #loss = tf.losses.softmax_cross_entropy(onehot_labels=labels, logits=logits)
            loss = tf.losses.sigmoid_cross_entropy(labels, logits)
            train_op = tf.contrib.layers.optimize_loss(
                loss=loss,
                global_step=tf.contrib.framework.get_global_step(),
                learning_rate=LEARNING_RATE,
                optimizer="SGD")
        if mode == tf.contrib.learn.ModeKeys.INFER:
            return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions)
        return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions, loss=loss, train_op=train_op)

    def train_model(self, tensors_to_log, reuse=False):
        if reuse:
            tsa_classifier = self.model
        else:
            tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                     model_dir=self.checkpoint_path + "/" + self.model_id)
        logging_hook = tf.train.LoggingTensorHook(tensors=tensors_to_log, every_n_iter=100)
        tsa_classifier.fit(
            x=InputImagesIterator(self.ids, self.data_path, 10000, self.y_slice, self.x_slice), 
            y=InputLabelsIterator(self.ids, self.labels), 
            steps=STEPS, 
            batch_size=BATCH_SIZE, 
            monitors=[logging_hook])
        self.model = tsa_classifier

    def load_model(self):
        print(self.checkpoint_path, self.model_id)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=self.checkpoint_path + "/" + self.model_id)
        print ("LOADED MODEL AT STEP " + str(tsa_classifier.get_variable_value("global_step")) + " FROM " + self.checkpoint_path + "/" + self.model_id )
        self.model = tsa_classifier

    def bootstrap_model(self, path=None):
        if path:
            checkpoint_path = path
        else:
            checkpoint_path = self.checkpoint_path + "/" + self.model_id
        print ("USING CHECKPOINT PATH  " + checkpoint_path)
        tsa_classifier = tf.contrib.learn.Estimator(model_fn=self.build_model, 
                                                    model_dir=checkpoint_path)
        print ("BOOTSTRAPPED AT STEP " + str(tsa_classifier.get_variable_value("global_step")))
        self.model = tsa_classifier 

    def predict(self):
        return self.model.predict(x=InputImagesIterator(self.ids, 
                                                            self.data_path, 
                                                            10000, 
                                                            self.y_slice,
                                                            self.x_slice, False))

In [71]:
image_df = pd.read_csv(DATA_PATH + '/stage1_labels.csv')
image_df['zone'] = image_df['Id'].str.split("_Zone", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()

labels = image_df[image_df['id'].isin(ids)]
labels = labels.sort_values("id")

tensors_to_log =  {"probabilities": "prob_tensor",
                    "actual":"labels"}
labels.zone = labels.zone.astype("int")
labels = labels.sort_values(["id","zone"])
labels = labels["Probability"]
labels = np.reshape(labels, (-1,17))

In [5]:
with tf.Session() as sess:
    model = ZoneModel(MODEL_ID, ids, "trimmed", "both", DATA_PATH, labels, CHECKPOINT_PATH, localize=False)     
    model.train_model(tensors_to_log, reuse=False)

INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce2c15e290>, '_model_dir': '/output/multi-class8/multi_class', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce2c15e290>, '_model_dir': '/output/multi-class8/multi_class', '_save_checkpoints_steps': None, 

NameError: global name 'InputImagesIterator' is not defined

In [76]:
def read_header(infile):
    """Read image header (first 512 bytes)
    """
    h = dict()
    fid = open(infile, 'rb')
    h['filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['parent_filename'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 20))
    h['comments1'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['comments2'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 80))
    h['energy_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['config_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['file_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['trans_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['date_modified'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 16))
    h['frequency'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['mat_velocity'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_polarization_channels'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare00'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['adc_min_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_max_voltage'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['band_width'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare01'] = np.fromfile(fid, dtype = np.int16, count = 5)
    h['polarization_type'] = np.fromfile(fid, dtype = np.int16, count = 4)
    h['record_header_size'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['word_precision'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['min_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['max_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['avg_data_value'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_scale_factor'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['data_units'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['surf_removal'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['edge_weighting'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['x_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['y_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['z_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['t_units'] = np.fromfile(fid, dtype = np.uint16, count = 1)
    h['spare02'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_return_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['scan_orientation'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scan_direction'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['data_storage_order'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['x_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_inc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['num_x_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_y_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_z_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['num_t_pts'] = np.fromfile(fid, dtype = np.int32, count = 1)
    h['x_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_speed'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_acc'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_motor_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_encoder_res'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['date_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['time_processed'] = b''.join(np.fromfile(fid, dtype = 'S1', count = 8))
    h['depth_recon'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['elevation_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['roll_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_max_travel'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['azimuth_offset_angle'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['adc_type'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['spare06'] = np.fromfile(fid, dtype = np.int16, count = 1)
    h['scanner_radius'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['x_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['y_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['z_offset'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['t_delay'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_start'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['range_gate_end'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['ahis_software_version'] = np.fromfile(fid, dtype = np.float32, count = 1)
    h['spare_end'] = np.fromfile(fid, dtype = np.float32, count = 10)
    return h


def read_data(infile, vertical="both", horizontal="both"):
    """Read any of the 4 types of image files, returns a numpy array of the image contents
    """
    extension = os.path.splitext(infile)[1]
    h = read_header(infile)
    nx = int(h['num_x_pts'])
    ny = int(h['num_y_pts'])
    nt = int(h['num_t_pts'])
    fid = open(infile, 'rb')
    fid.seek(512) #skip header
    if extension == '.aps' or extension == '.a3daps':
        if(h['word_type']==7): #float32
            data = np.fromfile(fid, dtype = np.float32, count = nx * ny * nt)
        elif(h['word_type']==4): #uint16
            data = np.fromfile(fid, dtype = np.uint16, count = nx * ny * nt)
        data = data * h['data_scale_factor'] #scaling factor
        data = data.reshape(nx, ny, nt, order='F').copy() #make N-d image
        if vertical == "bottom":
            data = data[:, :340, :] 
        elif vertical == "top":
            data = data[:, 320:660, :] 
        elif vertical == "middle":
            data = data[:, 170:510, :] 
        rotated_data = []
        if horizontal == "right":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[(i * increment) + 50:(i * increment) + 320,:,i])
        elif horizontal == "left":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[210 - (i * increment): 480-(i * increment):,:,i])
        elif horizontal == "middle":
            for i in range(0,16):
                increment = 10
                rotated_data.append(data[130:400,:,i])
        elif horizontal == "trimmed":
            for i in range(0,16):
                rotated_data.append(data[60:-60:,:,i])
        else:
            for i in range(0,16):
                rotated_data.append(data[:,:,i])
        data = np.array(rotated_data)  
    fid.close()
    return np.swapaxes(data, 2, 1)

class InputImagesIterator:
    def __init__(self, ids, data_path, contrast=1, vertical="both", horizontal="both", repeating=True):
        self.ids=ids
        self.contrast = contrast
        self.data_path=data_path
        self.i = -1
        self.vertical = vertical
        self.horizontal = horizontal
        self.repeating = repeating

    def __iter__(self):
        return self

    def next(self):
#         print ("id " + str(self.ids[self.i-1])) 
#         print("image iter " + str(self.i))
        if self.i < len(self.ids) -1:
            self.i = self.i + 1
            #print("IMAGES ITERATOR " + str(self.ids[self.i - 1]))
            return np.stack(read_data(self.data_path + self.ids[self.i] + ".aps", self.vertical, self.horizontal) * self.contrast)
        else:
            if not self.repeating:
                raise StopIteration()
            #Restart iteration, cycle back through
            self.i = -1
            #print("IMAGES ITERATOR " + str(self.ids[0]))
            return np.stack(read_data(self.data_path + self.ids[0] + ".aps", self.vertical, self.horizontal) * self.contrast)

class InputLabelsIterator:
    def __init__(self, ids, labels):
        self.ids=ids
        self.labels=labels
        self.i=-1

    def __iter__(self):
        return self

    def next(self):
#         print("in label iter " + str(self.ids[self.i -1])) 
        #print("in label iter " + str(self.test_labels[self.i -1])) 
#         print("label iter " + str(self.i))
        if self.i < len(self.ids) -1:
            self.i = self.i + 1
            #print("LABEL" + str(self.test_labels[self.i -1]))
            #return(self.labels[self.i] + EPSILON)
            return(self.labels[self.i])
        else:
            #Restart iteration, cycle back through
            self.i = -1
            #raise StopIteration()
            #return(self.labels[0] + EPSILON)
            return(self.labels[0])



In [84]:
image_df = pd.read_csv(DATA_PATH + 'stage1_sample_submission.csv')
image_df['zone'] = image_df['Id'].str.split("_", expand=True)[1].str.strip()
image_df['id'] = image_df['Id'].str.split("_", expand=True)[0].str.strip()

ids = image_df["id"].unique()
ids.sort()

csv_file = open(CHECKPOINT_PATH + "/submission.csv", "w+")
csv_file.write("Id,Probability\n")


model = ZoneModel(MODEL_ID, ids, "trimmed","both", DATA_PATH, image_df, '/output/', localize=False)
model.load_model()
print ("LOADED MODEL " + str(model))
predicted = np.array([x["probabilities"] for x in list(model.predict())])
for i, image in enumerate(predicted):
    for j, zone in enumerate(image):
        csv_file.write(str(ids[i]) + "_Zone" + str(j+1) + "," + str(zone) + "\n")
csv_file.close()



('/output/', 'multi_class')
INFO:tensorflow:Using default config.
Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce2437bcd0>, '_model_dir': '/output//multi_class', '_save_checkpoints_steps': None, '_keep_checkpoint_every_n_hours': 10000, '_session_config': None, '_tf_random_seed': None, '_environment': 'local', '_num_worker_replicas': 0, '_task_id': 0, '_save_summary_steps': 100, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_evaluation_master': '', '_master': ''}
Using config: {'_save_checkpoints_secs': 600, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_task_type': None, '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fce2437bcd0>, '_model_dir': '/output//multi_class', '_save_checkpoints_steps': No

From <ipython-input-70-da9f637f76a5>:107: calling predict (from tensorflow.contrib.learn.python.learn.estimators.estimator) with x is deprecated and will be removed after 2016-12-01.
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))


(392, 660)
Tensor("conv1/BiasAdd:0", shape=(1, 16, 660, 392, 12), dtype=float32)
Tensor("conv2/Relu:0", shape=(1, 16, 330, 196, 12), dtype=float32)
Tensor("pool1/MaxPool3D:0", shape=(1, 1, 164, 97, 12), dtype=float32)
Tensor("pool2/MaxPool:0", shape=(1, 81, 48, 12), dtype=float32)
Tensor("conv3/BiasAdd:0", shape=(1, 81, 48, 12), dtype=float32)
Tensor("pool1_2/MaxPool:0", shape=(1, 80, 47, 12), dtype=float32)
Tensor("conv4/Relu:0", shape=(1, 80, 47, 12), dtype=float32)
Tensor("pool2_2/MaxPool:0", shape=(1, 80, 47, 12), dtype=float32)
INFO:tensorflow:Restoring parameters from /output//multi_class/model.ckpt-3727
Restoring parameters from /output//multi_class/model.ckpt-3727
